In [1]:
import pandas as pd

from sklearn.preprocessing import OrdinalEncoder

In [2]:
df = pd.read_csv('../data/survey_data_before_pre_processing.csv')

### Remove Refused Persons

General rundown here is to calculate the amount of refusals per person as a percentage of total questions refused.

Find some cutoff and drop those people

### Convert all Refused to NaN

We will be imputing these, but need them to be nulls

### Ordinal Encode

There are a lot these.....

In [3]:
df.income_category.unique()

array(['$50,000 to $99,999', 'Less than $50,000', '$100,000 or more'],
      dtype=object)

In [4]:
cats = [['Greatest (Before 1928)','Silent (1928 - 1945)',
         'Baby Boomers (1946 - 1964)',
         'Generation X (1965 - 1980)',
         'Millennials (1981 - 1996)',
         'iGen\u200e/Gen Z (1997 - )'],
        ['Less than high school',
         'High school',
         'Some college',
         'Bachelor\'s degree or higher'],
        ['Less than $50,000',
         '$50,000 to $99,999',
         '$100,000 or more']]

o_enc = OrdinalEncoder(categories=cats)
o_enc.fit(df[['generation', 'educ_category', 'income_category']])
X_ord = o_enc.transform(df[['generation', 'educ_category', 'income_category']])
X_ord

array([[1., 1., 1.],
       [2., 0., 0.],
       [2., 3., 0.],
       ...,
       [4., 2., 2.],
       [3., 2., 0.],
       [4., 1., 0.]])

In [5]:
df[['generation', 'educ_category', 'income_category']]

,generation,educ_category,income_category
0,Silent (1928 - 1945),High school,"$50,000 to $99,999"
1,Baby Boomers (1946 - 1964),Less than high school,"Less than $50,000"
2,Baby Boomers (1946 - 1964),Bachelor's degree or higher,"Less than $50,000"
3,Silent (1928 - 1945),Bachelor's degree or higher,"$100,000 or more"
4,Millennials (1981 - 1996),Some college,"$50,000 to $99,999"
...,...,...,...
22340,Baby Boomers (1946 - 1964),Some college,"Less than $50,000"
22341,Baby Boomers (1946 - 1964),Less than high school,"$50,000 to $99,999"
22342,Millennials (1981 - 1996),Some college,"$100,000 or more"
22343,Generation X (1965 - 1980),Some college,"Less than $50,000"


### Train Test Split

In [7]:
from sklearn.model_selection import train_test_split

In [8]:
y = df.happening
X = df[[cols for cols in df.columns if cols != "happening"]]

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X,y,
                                                   test_size = 0.20,
                                                   random_state = 21)

### Iteratively Impute Nulls

Now all of our data is ordinal encoded or dummied, iteratively impute the nulls

In [14]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

In [15]:
# identify numerical columns
cols = X_train.describe().columns

In [16]:
# iterative impute on numerical columns
pd.DataFrame(IterativeImputer(initial_strategy='mean',
                              add_indicator=True,
                              random_state = 21
                             ).fit_transform(X_train[cols]))\
            .head(10)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,2017.0,0.6766,0.612152,0.0,50.0,6.0,3.0,0.0,0.0,0.0,1.0,2.0,1.0,0.0,1.0
1,2014.0,0.8173,0.756268,0.0,51.0,4.0,5.0,0.0,0.0,1.0,1.0,3.0,1.0,0.0,2.0
2,2012.0,0.2382,0.264869,1.0,48.0,2.0,2.0,0.0,0.0,0.0,0.0,2.0,1.0,0.0,0.0
3,2013.0,4.1649,4.702110,1.0,19.0,4.0,10.0,0.0,2.0,2.0,3.0,3.0,0.0,0.0,7.0
4,2014.0,0.6605,0.611177,1.0,56.0,1.0,5.0,0.0,2.0,0.0,0.0,3.0,0.0,1.0,2.0
5,2018.0,1.2250,1.130862,1.0,30.0,3.0,2.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0
6,2008.0,2.7200,1.482915,1.0,60.0,6.0,2.0,0.0,0.0,0.0,0.0,2.0,0.0,1.0,0.0
7,2008.0,0.7900,0.430699,1.0,49.0,5.0,4.0,0.0,0.0,0.0,0.0,4.0,1.0,0.0,0.0
8,2017.0,1.0797,0.976855,1.0,61.0,5.0,4.0,0.0,0.0,0.0,0.0,4.0,0.0,1.0,0.0
9,2013.0,0.3359,0.379226,0.0,80.0,1.0,2.0,0.0,0.0,0.0,0.0,2.0,0.0,1.0,0.0


### Standard Scale

Will let us explore knn or clustering if wanted